#Download & Unzip Data



In [ ]:
! cp /content/drive/MyDrive/DL/ADE20K_1.zip ./
! cp /content/drive/MyDrive/DL/ADE20K_2.zip ./
! cp /content/drive/MyDrive/DL/VOC2007.zip ./

In [ ]:
!mkdir ADE20K
!mkdir VOC2007
!unzip /content/ADE20K_1.zip -d ./ADE20K
!unzip /content/ADE20K_2.zip -d ./ADE20K
!unzip /content/VOC2007.zip -d ./VOC2007

In [ ]:
! rm /content/ADE20K/ADE20K/ADE20K_DL_course/imgs/ADE_val_00001280\(1\).jpg
! rm /content/ADE20K/ADE20K/ADE20K_DL_course/imgs/ADE_val_00001281\(1\).jpg
! rm /content/ADE20K/ADE20K/ADE20K_DL_course/imgs/ADE_val_00001282\(1\).jpg
! rm /content/ADE20K/ADE20K/ADE20K_DL_course/imgs/ADE_val_00001283\(1\).jpg
! rm /content/ADE20K/ADE20K/ADE20K_DL_course/imgs/ADE_val_00001285\(1\).jpg

# Starter Code ADE20K
This notebook contains a tutorial on how to explore data in ADE20K

##import

In [ ]:
from PIL import Image
import matplotlib._color_data as mcd
import cv2
import json
import numpy as np
import os

_NUMERALS = '0123456789abcdefABCDEF'
_HEXDEC = {v: int(v, 16) for v in (x+y for x in _NUMERALS for y in _NUMERALS)}
LOWERCASE, UPPERCASE = 'x', 'X'
def rgb(triplet):
    return _HEXDEC[triplet[0:2]], _HEXDEC[triplet[2:4]], _HEXDEC[triplet[4:6]]

def loadAde20K(file):
    fileseg = "/content/ADE20K/ADE20K/ADE20K_DL_course/masks/"+file.replace('.jpg', '_seg.png').split("/")[-1];
    with Image.open(fileseg) as io:
        seg = np.array(io);

    # Obtain the segmentation mask, bult from the RGB channels of the _seg file
    R = seg[:,:,0];
    G = seg[:,:,1];
    B = seg[:,:,2];
    ObjectClassMasks = (R/10).astype(np.int32)*256+(G.astype(np.int32));


    # Obtain the instance mask from the blue channel of the _seg file
    Minstances_hat = np.unique(B, return_inverse=True)[1]
    Minstances_hat = np.reshape(Minstances_hat, B.shape)
    ObjectInstanceMasks = Minstances_hat


    level = 0
    PartsClassMasks = [];
    PartsInstanceMasks = [];
    while True:
        level = level+1;
        file_parts = file.replace('.jpg', f'_parts_{level:03}.png');
        if os.path.isfile(file_parts):
            with Image.open(file_parts) as io:
                partsseg = np.array(io);
            R = partsseg[:,:,0];
            G = partsseg[:,:,1];
            B = partsseg[:,:,2];
            PartsClassMasks.append((np.int32(R)/10)*256+np.int32(G));
            PartsInstanceMasks = PartsClassMasks
            # TODO:  correct partinstancemasks


        else:
            break

    objects = {}
    parts = {}

    attr_file_name = "/content/ADE20K/ADE20K/ADE20K_DL_course/jsons/" + file.replace('.jpg', '.json').split("/")[-1];
    if os.path.isfile(attr_file_name):
        with open(attr_file_name, 'r') as f:
            input_info = json.load(f)

        contents = input_info['annotation']['object']
        instance = np.array([int(x['id']) for x in contents])
        names = [x['raw_name'] for x in contents]
        corrected_raw_name =  [x['name'] for x in contents]
        partlevel = np.array([int(x['parts']['part_level']) for x in contents])
        ispart = np.array([p>0 for p in partlevel])
        iscrop = np.array([int(x['crop']) for x in contents])
        listattributes = [x['attributes'] for x in contents]
        polygon = [x['polygon'] for x in contents]
        for p in polygon:
            p['x'] = np.array(p['x'])
            p['y'] = np.array(p['y'])

        objects['instancendx'] = instance[ispart == 0]
        objects['class'] = [names[x] for x in list(np.where(ispart == 0)[0])]
        objects['corrected_raw_name'] = [corrected_raw_name[x] for x in list(np.where(ispart == 0)[0])]
        objects['iscrop'] = iscrop[ispart == 0]
        objects['listattributes'] = [listattributes[x] for x in list(np.where(ispart == 0)[0])]
        objects['polygon'] = [polygon[x] for x in list(np.where(ispart == 0)[0])]


        parts['instancendx'] = instance[ispart == 1]
        parts['class'] = [names[x] for x in list(np.where(ispart == 1)[0])]
        parts['corrected_raw_name'] = [corrected_raw_name[x] for x in list(np.where(ispart == 1)[0])]
        parts['iscrop'] = iscrop[ispart == 1]
        parts['listattributes'] = [listattributes[x] for x in list(np.where(ispart == 1)[0])]
        parts['polygon'] = [polygon[x] for x in list(np.where(ispart == 1)[0])]

    return {'img_name': file, 'segm_name': fileseg,
            'class_mask': ObjectClassMasks, 'instance_mask': ObjectInstanceMasks,
            'partclass_mask': PartsClassMasks, 'part_instance_mask': PartsInstanceMasks,
            'objects': objects, 'parts': parts}

def plot_polygon(img_name, info, show_obj=True, show_parts=False):
    colors = mcd.CSS4_COLORS
    color_keys = list(colors.keys())
    all_objects = []
    all_poly = []
    if show_obj:
        all_objects += info['objects']['class']
        all_poly += info['objects']['polygon']
    if show_parts:
        all_objects += info['parts']['class']
        all_poly += info['objects']['polygon']

    img = cv2.imread(img_name)
    thickness = 5
    for it, (obj, poly) in enumerate(zip(all_objects, all_poly)):
        curr_color = colors[color_keys[it % len(color_keys)] ]
        pts = np.concatenate([poly['x'][:, None], poly['y'][:, None]], 1)[None, :]
        color = rgb(curr_color[1:])
        img = cv2.polylines(img, pts, True, color, thickness)
    return img


In [ ]:
%load_ext autoreload
%autoreload 2
import IPython.display
import matplotlib.pyplot as plt
import cv2
import numpy as np
import pickle as pkl

## Dataset index

In [ ]:
# Load index with global information about ADE20K
DATASET_PATH = '/content/ADE20K/ADE20K/ADE20K_DL_course'
index_file = 'index_ade20k.pkl'
with open('{}/{}'.format(DATASET_PATH, index_file), 'rb') as f:
    index_ade20k = pkl.load(f)

In [ ]:
objectIsPart = []
for i in range(len(index_ade20k["objectIsPart"])):
  objectIsPart.append(index_ade20k["objectIsPart"][i][25258:27258])
objectIsPart = np.array(objectIsPart)

In [ ]:
objectPresence = []
for i in range(len(index_ade20k["objectPresence"])):
  objectPresence.append(index_ade20k["objectPresence"][i][25258:27258])
objectPresence = np.array(objectPresence)

In [ ]:
new = {}
new["filename"] = index_ade20k["filename"][25258:27258]
new["objectIsPart"] = objectIsPart
new["objectPresence"] = objectPresence
new["objectcounts"] = index_ade20k["objectcounts"]
new["objectnames"] = index_ade20k["objectnames"]
new["scene"] = index_ade20k["scene"][25258:27258]
new["wordnet_found"] = index_ade20k["wordnet_found"]
new["wordnet_level1"] = index_ade20k["wordnet_level1"]
new["wordnet_synset"] = index_ade20k["wordnet_synset"]
new["wordnet_hypernym"] = index_ade20k["wordnet_hypernym"]
new["wordnet_gloss"] = index_ade20k["wordnet_gloss"]
new["wordnet_frequency"] = index_ade20k["wordnet_frequency"]
new["description"] = index_ade20k["description"]

In [ ]:
print(new["filename"][0])
print(new["filename"][1999])

In [ ]:
# print("File loaded, description of the attributes:")
# print('--------------------------------------------')
# for attribute_name, desc in new['description'].items():
#     print('* {}: {}'.format(attribute_name, desc))
# print('--------------------------------------------\n')

ADE = {}
for i in range(2000):
  ADE[i] = {}
  nfiles = len(new['filename'])
  ADE[i]["file_name"] = new['filename'][i]
  ADE[i]["num_obj"] = new['objectPresence'][:, i].sum()
  ADE[i]["num_parts"] = new['objectIsPart'][:, i].sum()
  count_obj = new['objectPresence'][:, i].max()
  obj_id = np.where(new['objectPresence'][:, i] == count_obj)[0][0]
  obj_name = new['objectnames'][obj_id]
  full_file_name = '{}/{}'.format("imgs", new['filename'][i])
  ADE[i]["count_obj"] = count_obj
  ADE[i]["obj_id"] = obj_id
  ADE[i]["obj_name"] = obj_name
  ADE[i]["full_file_name"] = full_file_name

print("The dataset has {} images".format(nfiles))
print("The image at index {} is {}".format(i, ADE[i]["file_name"]))
print("It is located at {}".format(full_file_name))
print("It happens in a {}".format(new['scene'][i]))
print("It has {} objects, of which {} are parts".format(ADE[i]["num_obj"], ADE[i]["num_parts"]))
print("The most common object is object {} ({}), which appears {} times".format(obj_name, obj_id, count_obj))

In [ ]:
root_path = DATASET_PATH

# This function reads the image and mask files and generate instance and segmentation
# masks

i = 1999
info = loadAde20K('{}/{}'.format(root_path, ADE[i]["full_file_name"]))
img = cv2.imread(info['img_name'])[:,:,::-1]
seg = cv2.imread(info['segm_name'])[:,:,::-1]

In [ ]:
info["objects"]

You can also inspect the attributes `info['objects']` and `info['parts']` for information about object names, attributes etc.

#VOC 2007

In [ ]:
import torch
root_train_imgs = "/content/ADE20K/ADE20K/ADE20K_DL_course/imgs/"
root_train_mask = "/content/ADE20K/ADE20K/ADE20K_DL_course/masks/"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
import cv2 as cv
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, datasets, models

class RoadDataset(Dataset):

    def __init__(self, width, height, path_to_imgs, path_to_mask, transform = None):

        self.height = height
        self.width = width
        self.path_to_img = path_to_imgs
        self.path_to_mask= path_to_mask

        self.train_imgs = os.listdir(path_to_imgs)
        self.train_mask = os.listdir(path_to_mask)

        self.length = len(self.train_imgs)
        self.transform = transform

    def __getitem__(self, index):

        img = cv.imread(self.path_to_img + self.train_imgs[index])
        msk = cv.imread(self.path_to_mask + self.train_mask[index])

        img_resize = cv.resize(img, (self.width, self.height), interpolation = cv.INTER_CUBIC)
        msk_resize = cv.resize(msk, (self.width, self.height), interpolation = cv.INTER_NEAREST)

        msk_transpose = msk_resize.transpose((2, 0, 1))
        msk_one_channel = msk_transpose[0]

        if self.transform:
            img_tensor = self.transform(img_resize)

        return (img_tensor, msk_one_channel)


    def __len__(self):
        return self.length

# This will normalize the image value
trans = transforms.Compose([
    transforms.ToTensor(),
])

# Set DataLoader
width = 256
height = 256
batch_size = 12

Train_Dataset = RoadDataset(width, height, root_train_imgs, root_train_mask, trans)
Train_Dataloader = DataLoader(Train_Dataset, batch_size = batch_size, shuffle = True, num_workers = 0)

#Model

backbone(ResNet50)

In [ ]:
import tensorflow as tf
from tensorflow.python.training import moving_averages

fc_initializer = tf.keras.initializers.glorot_normal
conv2d_initializer = tf.keras.initializers.GlorotNormal

# create weight variable
def create_var(name, shape, initializer, trainable=True):
    return tf.compat.v1.get_variable(name, shape=shape, dtype=tf.float32,
                           initializer=initializer, trainable=trainable)

# conv2d layer
def conv2d(x, num_outputs, kernel_size, stride=1, scope="conv2d"):
    num_inputs = x.get_shape()[-1]
    with tf.compat.v1.variable_scope(scope):
        kernel = create_var("kernel", [kernel_size, kernel_size,
                                       num_inputs, num_outputs],
                            conv2d_initializer())
        return tf.nn.conv2d(x, kernel, strides=[1, stride, stride, 1],
                            padding="SAME")

# fully connected layer
def fc(x, num_outputs, scope="fc"):
    num_inputs = x.get_shape()[-1]
    with tf.compat.v1.variable_scope(scope):
        weight = create_var("weight", [num_inputs, num_outputs],
                            fc_initializer())
        bias = create_var("bias", [num_outputs,],
                          tf.zeros_initializer())
        return tf.compat.v1.nn.xw_plus_b(x, weight, bias)


# batch norm layer
def batch_norm(x, decay=0.999, epsilon=1e-03, is_training=True,
               scope="scope"):
    x_shape = x.get_shape()
    num_inputs = x_shape[-1]
    reduce_dims = list(range(len(x_shape) - 1))
    with tf.compat.v1.variable_scope(scope):
        beta = create_var("beta", [num_inputs,],
                               initializer=tf.zeros_initializer())
        gamma = create_var("gamma", [num_inputs,],
                                initializer=tf.ones_initializer())
        # for inference
        moving_mean = create_var("moving_mean", [num_inputs,],
                                 initializer=tf.zeros_initializer(),
                                 trainable=False)
        moving_variance = create_var("moving_variance", [num_inputs],
                                     initializer=tf.ones_initializer(),
                                     trainable=False)
    if is_training:
        mean, variance = tf.nn.moments(x, axes=reduce_dims)
        update_move_mean = moving_averages.assign_moving_average(moving_mean,
                                                mean, decay=decay)
        update_move_variance = moving_averages.assign_moving_average(moving_variance,
                                                variance, decay=decay)
        tf.compat.v1.add_to_collection(tf.compat.v1.GraphKeys.UPDATE_OPS, update_move_mean)
        tf.compat.v1.add_to_collection(tf.compat.v1.GraphKeys.UPDATE_OPS, update_move_variance)
    else:
        mean, variance = moving_mean, moving_variance
    return tf.nn.batch_normalization(x, mean, variance, beta, gamma, epsilon)


# avg pool layer
def avg_pool(x, pool_size, scope):
    with tf.compat.v1.variable_scope(scope):
        return tf.nn.avg_pool(x, [1, pool_size, pool_size, 1],
                strides=[1, pool_size, pool_size, 1], padding="VALID")

# max pool layer
def max_pool(x, pool_size, stride, scope):
    with tf.compat.v1.variable_scope(scope):
        return tf.nn.max_pool(x, [1, pool_size, pool_size, 1],
                              [1, stride, stride, 1], padding="SAME")

class ResNet50(object):
    def __init__(self, inputs, num_classes=1000, is_training=True,
                 scope="resnet50"):
        self.inputs =inputs
        self.is_training = is_training
        self.num_classes = num_classes

        with tf.compat.v1.variable_scope(scope):
            # construct the model
            net = conv2d(inputs, 64, 7, 2, scope="conv1") # -> [batch, 112, 112, 64]
            net = tf.nn.relu(batch_norm(net, is_training=self.is_training, scope="bn1"))
            net = max_pool(net, 3, 2, scope="maxpool1")  # -> [batch, 56, 56, 64]
            net = self._block(net, 256, 3, init_stride=1, is_training=self.is_training,
                              scope="block2")           # -> [batch, 56, 56, 256]
            net = self._block(net, 512, 4, is_training=self.is_training, scope="block3")
                                                        # -> [batch, 28, 28, 512]
            net = self._block(net, 1024, 6, is_training=self.is_training, scope="block4")
                                                        # -> [batch, 14, 14, 1024]
            net = self._block(net, 2048, 3, is_training=self.is_training, scope="block5")
                                                        # -> [batch, 7, 7, 2048]
            net = avg_pool(net, 7, scope="avgpool5")    # -> [batch, 1, 1, 2048]
            net = tf.squeeze(net, [1, 2], name="SpatialSqueeze") # -> [batch, 2048]
            self.logits = fc(net, self.num_classes, "fc6")       # -> [batch, num_classes]
            self.predictions = tf.nn.softmax(self.logits)


    def _block(self, x, n_out, n, init_stride=2, is_training=True, scope="block"):
        with tf.compat.v1.variable_scope(scope):
            h_out = n_out // 4
            out = self._bottleneck(x, h_out, n_out, stride=init_stride,
                                   is_training=is_training, scope="bottlencek1")
            for i in range(1, n):
                out = self._bottleneck(out, h_out, n_out, is_training=is_training,
                                       scope=("bottlencek%s" % (i + 1)))
            return out

    def _bottleneck(self, x, h_out, n_out, stride=None, is_training=True, scope="bottleneck"):
        """ A residual bottleneck unit"""
        n_in = x.get_shape()[-1]
        if stride is None:
            stride = 1 if n_in == n_out else 2

        with tf.compat.v1.variable_scope(scope):
            h = conv2d(x, h_out, 1, stride=stride, scope="conv_1")
            h = batch_norm(h, is_training=is_training, scope="bn_1")
            h = tf.nn.relu(h)
            h = conv2d(h, h_out, 3, stride=1, scope="conv_2")
            h = batch_norm(h, is_training=is_training, scope="bn_2")
            h = tf.nn.relu(h)
            h = conv2d(h, n_out, 1, stride=1, scope="conv_3")
            h = batch_norm(h, is_training=is_training, scope="bn_3")

            if n_in != n_out:
                shortcut = conv2d(x, n_out, 1, stride=stride, scope="conv_4")
                shortcut = batch_norm(shortcut, is_training=is_training, scope="bn_4")
            else:
                shortcut = x
            return tf.nn.relu(shortcut + h)


#test
if __name__ == "__main__":
    x = tf.random.normal([32, 224, 224, 3])
    resnet50 = ResNet50(x)
    print(resnet50.logits)

bottleneck_Object_detection (YOLO)

In [ ]:
import os

import numpy as np
import tensorflow as tf



######## basic layers #######

def leaky_relu(x):
    return tf.nn.leaky_relu(x, alpha=0.1, name="leaky_relu")

# Conv2d
def conv2d(x, filters, size, pad=0, stride=1, batch_normalize=1,
           activation=leaky_relu, use_bias=False, name="conv2d"):
    if pad > 0:
        x = tf.pad(x, [[0, 0], [pad, pad], [pad, pad], [0, 0]])
    out = tf.keras.layers.Conv2D(filters, size, strides=stride, padding="VALID",
                           activation=None, use_bias=use_bias, name=name)(x)
    if batch_normalize == 1:
        out = tf.compat.v1.layers.batch_normalization(out, axis=-1, momentum=0.9,
                                            training=False, name=name+"_bn")
    if activation:
        out = activation(out)
    return out

# maxpool2d
def maxpool(x, size=2, stride=2, name="maxpool"):
    return tf.compat.v1.layers.max_pooling2d(x, size, stride)

# reorg layer
def reorg(x, stride):
    return tf.compat.v1.extract_image_patches(x, [1, stride, stride, 1],
                        [1, stride, stride, 1], [1,1,1,1], padding="VALID")


def darknet(images, n_last_channels=425):
    """Darknet19 for YOLOv2"""
    net = conv2d(images, 32, 3, 1, name="conv1")
    net = maxpool(net, name="pool1")
    net = conv2d(net, 64, 3, 1, name="conv2")
    net = maxpool(net, name="pool2")
    net = conv2d(net, 128, 3, 1, name="conv3_1")
    net = conv2d(net, 64, 1, name="conv3_2")
    net = conv2d(net, 128, 3, 1, name="conv3_3")
    net = maxpool(net, name="pool3")
    net = conv2d(net, 256, 3, 1, name="conv4_1")
    net = conv2d(net, 128, 1, name="conv4_2")
    net = conv2d(net, 256, 3, 1, name="conv4_3")
    net = maxpool(net, name="pool4")
    net = conv2d(net, 512, 3, 1, name="conv5_1")
    net = conv2d(net, 256, 1, name="conv5_2")
    net = conv2d(net, 512, 3, 1, name="conv5_3")
    net = conv2d(net, 256, 1, name="conv5_4")
    net = conv2d(net, 512, 3, 1, name="conv5_5")
    shortcut = net
    net = maxpool(net, name="pool5")
    net = conv2d(net, 1024, 3, 1, name="conv6_1")
    net = conv2d(net, 512, 1, name="conv6_2")
    net = conv2d(net, 1024, 3, 1, name="conv6_3")
    net = conv2d(net, 512, 1, name="conv6_4")
    net = conv2d(net, 1024, 3, 1, name="conv6_5")
    # ---------
    net = conv2d(net, 1024, 3, 1, name="conv7_1")
    net = conv2d(net, 1024, 3, 1, name="conv7_2")
    # shortcut
    shortcut = conv2d(shortcut, 64, 1, name="conv_shortcut")
    shortcut = reorg(shortcut, 2)
    net = tf.concat([shortcut, net], axis=-1)
    net = conv2d(net, 1024, 3, 1, name="conv8")
    # detection layer
    net = conv2d(net, n_last_channels, 1, batch_normalize=0,
                 activation=None, use_bias=True, name="conv_dec")
    return net


#test
if __name__ == "__main__":
    x = tf.random.normal([1, 416, 416, 3])
    model = darknet(x)
    print(model)


bottleneck_semantic_segmentation (U-Net)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
import time

def train_model(model, optimizer, scheduler, num_epochs):
    probe_number = 200
    criterion = nn.CrossEntropyLoss()
    loss_list = []

    for epoch in range(num_epochs):

        since = time.time()

        for param_group in optimizer.param_groups:
            print("LR", param_group['lr'])
            model.train()

        inner_epoch_count = 0
        running_loss = 0

        for inputs, labels in tqdm(Train_Dataloader):

            inputs, labels = inputs.to(device), labels.to(device).long()

            optimizer.zero_grad()

            outputs = model(inputs)

            loss = criterion(outputs, labels)

            #epoch_samples += inputs.size(0)
            running_loss = running_loss + loss.item()
            inner_epoch_count = inner_epoch_count + 1
            if inner_epoch_count % probe_number == probe_number - 1:
                # Plot predict mask
                pred_np = outputs.data.cpu().numpy()[0]
                pred_argmax = np.argmax(pred_np, axis = 0)
                print(f"Prediction {np.unique(pred_argmax)}")
                plt.imshow(pred_argmax/10)
                plt.show()

                # Plot ground truth mask
                lab_np = labels.data.cpu().numpy()[0]
                print(f"Answer: {np.unique(lab_np)}")
                plt.imshow(lab_np/10)
                plt.show()

                # Plot input image
                inp_np = inputs.data.cpu().numpy()[0].transpose((1, 2, 0))
                plt.imshow(inp_np)
                plt.show()
                print("============================")
                print(f"[Epoch {epoch}/{num_epochs - 1}] Running loss: {round(running_loss / probe_number, 3)}")
                loss_list.append(running_loss / probe_number)
                running_loss = 0
                print("============================")

            loss.backward()

            optimizer.step()

        scheduler.step()

        time_elapsed = time.time() - since
        print('{:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))

    return model, loss_list

In [ ]:
import torch
import torch.nn as nn

def double_conv(in_channels, out_channels):
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, 3, padding=1),
        nn.ReLU(inplace=True),
        nn.Conv2d(out_channels, out_channels, 3, padding=1),
        nn.ReLU(inplace=True)
    )


class UNet(nn.Module):

    def __init__(self, n_class):
        super().__init__()

        self.dconv_down1 = double_conv(3, 64)
        self.dconv_down2 = double_conv(64, 128)
        self.dconv_down3 = double_conv(128, 256)
        self.dconv_down4 = double_conv(256, 512)

        self.maxpool = nn.MaxPool2d(2)
        self.upsample = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)

        self.dconv_up3 = double_conv(256 + 512, 256)
        self.dconv_up2 = double_conv(128 + 256, 128)
        self.dconv_up1 = double_conv(128 + 64, 64)

        self.conv_last = nn.Conv2d(64, n_class, 1)


    def forward(self, x):
        conv1 = self.dconv_down1(x)
        x = self.maxpool(conv1)

        conv2 = self.dconv_down2(x)
        x = self.maxpool(conv2)

        conv3 = self.dconv_down3(x)
        x = self.maxpool(conv3)

        x = self.dconv_down4(x)

        x = self.upsample(x)
        x = torch.cat([x, conv3], dim=1)

        x = self.dconv_up3(x)
        x = self.upsample(x)
        x = torch.cat([x, conv2], dim=1)

        x = self.dconv_up2(x)
        x = self.upsample(x)
        x = torch.cat([x, conv1], dim=1)

        x = self.dconv_up1(x)

        out = self.conv_last(x)

        return out

In [ ]:
#test
import torch
import torch.optim as optim
from torch.optim import lr_scheduler
import time
import copy

num_class = 10
num_epochs = 150

model = UNet(num_class).to(device)

optimizer_ft = optim.Adam(model.parameters(), lr = 1e-4)

exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size = 25, gamma = 0.1)

model, loss_list = train_model(model, optimizer_ft, exp_lr_scheduler, num_epochs = num_epochs)
